In [9]:
import carla 
import math 
import random 
import time 
import numpy as np
import cv2
import open3d as o3d
from matplotlib import cm
import os

from util.sensor_cb import camera_callback, lidar_callback, add_open3d_axis

In [10]:
# Connect the client and set up bp library and spawn point
client = carla.Client('localhost', 2000)
world = client.get_world()
bp_lib = world.get_blueprint_library() 
spawn_points = world.get_map().get_spawn_points() 

In [25]:
init_trans = carla.Transform(carla.Location(z=45, x=-80, y=2), carla.Rotation(pitch=-90, yaw=-90, roll=0))
spectator = world.get_spectator() 
spectator.set_transform(init_trans)

In [22]:
# 摆放3辆车
vehicle_bp = bp_lib.find('vehicle.mercedes-benz.coupe') 
vehicle_init_trans = carla.Transform(carla.Location(z=1, x=-78, y=10), carla.Rotation(pitch=0, yaw=-110, roll=0))
vehicle1 = world.spawn_actor(vehicle_bp, vehicle_init_trans)
# 2
vehicle_bp = bp_lib.find('vehicle.carlamotors.carlacola') 
vehicle_init_trans = carla.Transform(carla.Location(z=1, x=-84, y=-3), carla.Rotation(pitch=0, yaw=60, roll=0))
vehicle2 = world.spawn_actor(vehicle_bp, vehicle_init_trans)
# 3 
vehicle_bp = bp_lib.find('vehicle.audi.a2') 
vehicle_init_trans = carla.Transform(carla.Location(z=1, x=-88, y=-10), carla.Rotation(pitch=0, yaw=90, roll=0))
vehicle3 = world.spawn_actor(vehicle_bp, vehicle_init_trans)

In [5]:
# Set up LIDAR and RADAR, parameters are to assisst visualisation
lidar_bp = bp_lib.find('sensor.lidar.ray_cast') 
lidar_bp.set_attribute('range', '50')
lidar_bp.set_attribute('noise_stddev', '0.1')
lidar_bp.set_attribute('upper_fov', '4')
lidar_bp.set_attribute('lower_fov', '-20')
lidar_bp.set_attribute('channels', '64')
lidar_bp.set_attribute('rotation_frequency', '20')
lidar_bp.set_attribute('points_per_second', '2304000')

camera_bp = bp_lib.find('sensor.camera.rgb') 

In [6]:
# Set up vehicle sensors
vehicle_init_trans = carla.Transform(carla.Location(0, 0, 2), carla.Rotation(0, 0, 0))
lidar1 = world.spawn_actor(lidar_bp, vehicle_init_trans, attach_to=vehicle1)
camera1 = world.spawn_actor(camera_bp, vehicle_init_trans, attach_to=vehicle1)

lidar3 = world.spawn_actor(lidar_bp, vehicle_init_trans, attach_to=vehicle3)

camera_top = world.spawn_actor(camera_bp, init_trans)

point_list_vehicle1 = o3d.geometry.PointCloud()
point_list_vehicle3 = o3d.geometry.PointCloud()
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()
camera_data_vehicle = {'image': np.zeros((image_h, image_w, 4))} 

In [7]:
# Start drone sensors
lidar1.listen(lambda data: lidar_callback(data, point_list_vehicle1, f'output/left_turn/lidar1'))
camera1.listen(lambda image: camera_callback(image, camera_data_vehicle, f'output/left_turn/camera1'))
lidar3.listen(lambda data: lidar_callback(data, point_list_vehicle3, f'output/left_turn/lidar3'))

In [8]:
lidar1.stop()
camera1.stop()
lidar3.stop()

In [ ]:
# 现在的问题就是，我一直觉得点云图怪怪的，坐标系不对吗？
# 应该怎么保存数据？应该怎么查看数据？

In [ ]:
# OpenCV window for camera
cv2.namedWindow(f'RGB Camera (z={z}, x={x}, pitch={pitch}, roll={roll})', cv2.WINDOW_AUTOSIZE)
cv2.imshow(f'RGB Camera (z={z}, x={x}, pitch={pitch}, roll={roll})', camera_data_drone['image'])
cv2.waitKey(1)

# Open3D visualiser for LIDAR and RADAR
vis = o3d.visualization.Visualizer()
vis.create_window(
    window_name=f'Lidar (z={z}, x={x}, pitch={pitch}, roll={roll})',
    width=960,
    height=540,
    left=480,
    top=270)
vis.get_render_option().background_color = [0.05, 0.05, 0.05]
vis.get_render_option().point_size = 1
vis.get_render_option().show_coordinate_frame = True
add_open3d_axis(vis)

# Update geometry and camera in game loop
frame = 0
while True:
    if frame == 2:
        vis.add_geometry(point_list_drone)
    vis.update_geometry(point_list_drone)
    
    vis.poll_events()
    vis.update_renderer()
    # # This can fix Open3D jittering issues:
    time.sleep(0.005)
    frame += 1

    cv2.imshow(f'RGB Camera (z={z}, x={x}, pitch={pitch}, roll={roll})', camera_data_drone['image'])
    
    # Break if user presses 'q'
    if cv2.waitKey(1) == ord('q'):
        break

# Close displayws and stop sensors
cv2.destroyAllWindows()
lidar_drone.stop()
lidar_drone.destroy()
camera_drone.stop()
camera_drone.destroy()
vis.destroy_window()

# for actor in world.get_actors().filter('*vehicle*'):
#     actor.destroy()
# for actor in world.get_actors().filter('*sensor*'):
#     actor.destroy()

In [21]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for actor in world.get_actors().filter('*sensor*'):
    actor.destroy()